<a href="https://colab.research.google.com/github/count-im/test/blob/main/Python/Py05/Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import random


class Character:
    def __init__(self, name, level, hp, attack, defense):
        self.name = name
        self.level = level
        self.hp = hp
        self.attack = attack
        self.defense = defense

    def is_alive(self):
        return self.hp > 0

    def take_damage(self, damage):
        actual_damage = damage - self.defense
        if actual_damage > 0:
            self.hp -= actual_damage

    def attack_target(self, target):
        damage = random.randint(1, self.attack)
        print(f"{self.name}이 {target.name}에게 {damage} 만큼 공격했다...!")
        target.take_damage(damage)
        print(f"{target.name}의 체력: {target.hp}")


class Player(Character):
    def __init__(self, name):
        super().__init__(name=name, level=1, hp=100, attack=25, defense=5)
        self.exp = 0

    def gain_experience(self, amount):
        self.exp += amount

    def level_up(self):
        while self.exp >= 500:
            self.exp -= 500
            self.level += 1
            self.attack += 10
            self.defense += 5


class Monster(Character):
    def __init__(self, name, level):
        hp = random.randint(10, 30) * level
        attack = random.randint(5, 20) * level
        defense = random.randint(1, 5) * level
        super().__init__(name=name, level=level, hp=hp, attack=attack, defense=defense)


def battle(player, monster):
    print(f"{monster.name}과의 전투를 시작합니다.")
    while player.is_alive() and monster.is_alive():
        player.attack_target(monster)
        if not monster.is_alive():
            break
        monster.attack_target(player)

    if player.is_alive():
        player.gain_experience(monster.level * 20)
        player.level_up()
        print("전투 승리!")
        return True
    else:
        print("전투 패배...")
        return False


def main():
    monster_dict = {"슬라임": 1, "고블린": 2, "오크": 3}

    player_name = input("플레이어의 이름을 입력하세요: ")
    player = Player(player_name)

    for monster_name, monster_level in monster_dict.items():
        monster = Monster(monster_name, monster_level)
        win = battle(player, monster)

        if not win:
            print("게임오버")
            break
        else:
            print(f"{monster.name}을 이겼다!")


if __name__ == "__main__":
    main()



플레이어의 이름을 입력하세요: 0000
슬라임과의 전투를 시작합니다.
0000이 슬라임에게 23 만큼 공격했다...!
슬라임의 체력: -4
전투 승리!
슬라임을 이겼다!
고블린과의 전투를 시작합니다.
0000이 고블린에게 11 만큼 공격했다...!
고블린의 체력: 55
고블린이 0000에게 2 만큼 공격했다...!
0000의 체력: 100
0000이 고블린에게 12 만큼 공격했다...!
고블린의 체력: 51
고블린이 0000에게 15 만큼 공격했다...!
0000의 체력: 90
0000이 고블린에게 10 만큼 공격했다...!
고블린의 체력: 49
고블린이 0000에게 30 만큼 공격했다...!
0000의 체력: 65
0000이 고블린에게 23 만큼 공격했다...!
고블린의 체력: 34
고블린이 0000에게 18 만큼 공격했다...!
0000의 체력: 52
0000이 고블린에게 3 만큼 공격했다...!
고블린의 체력: 34
고블린이 0000에게 27 만큼 공격했다...!
0000의 체력: 30
0000이 고블린에게 13 만큼 공격했다...!
고블린의 체력: 29
고블린이 0000에게 13 만큼 공격했다...!
0000의 체력: 22
0000이 고블린에게 5 만큼 공격했다...!
고블린의 체력: 29
고블린이 0000에게 17 만큼 공격했다...!
0000의 체력: 10
0000이 고블린에게 14 만큼 공격했다...!
고블린의 체력: 23
고블린이 0000에게 17 만큼 공격했다...!
0000의 체력: -2
전투 패배...
게임오버


업그레이드 목표

전투 로그 강화

공격 결과(치명타/회피/실제 데미지/방어로 상쇄)를 더 명확히 출력

치명타(Critical) / 회피(Evasion)

Player/Monster 모두 확률로 치명타, 회피 발생

스킬 2개 (플레이어 전용)

power_strike: 강공(높은 데미지, 쿨다운)

guard: 방어 태세(해당 턴 받는 피해 감소)

포션(소모품) + 인벤토리

전투 중 포션 사용 가능 (자동/수동 선택 가능)

레벨업 확장

레벨업 시 공격/방어 증가 + HP 최대치도 증가 + 전투 후 체력 일부 회복(“숨 고르기”)

In [4]:
import random


# -----------------------------
# Core Entities
# -----------------------------
class Character:
    def __init__(self, name, level, hp, attack, defense, crit=0.05, evasion=0.05):
        self.name = name
        self.level = level

        self.max_hp = hp
        self.hp = hp

        self.attack = attack
        self.defense = defense

        self.crit = crit        # 치명타 확률 (0~1)
        self.evasion = evasion  # 회피 확률 (0~1)

        # 상태(버프/디버프)
        self.guard_until_next_turn = False

    def is_alive(self):
        return self.hp > 0

    def _try_evade(self):
        return random.random() < self.evasion

    def _try_crit(self):
        return random.random() < self.crit

    def take_damage(self, raw_damage):
        # 회피 체크
        if self._try_evade():
            print(f"  - {self.name}이(가) 회피했다!")
            return 0

        # 방어 태세: 다음 1회 피격 피해 감소 (간단 구현)
        if self.guard_until_next_turn:
            raw_damage = int(raw_damage * 0.6)  # 40% 경감
            self.guard_until_next_turn = False
            print(f"  - {self.name}의 방어 태세로 피해가 감소했다!")

        actual_damage = raw_damage - self.defense
        if actual_damage <= 0:
            print(f"  - {self.name}은(는) 방어로 피해를 상쇄했다! (0)")
            return 0

        self.hp -= actual_damage
        if self.hp < 0:
            self.hp = 0

        print(f"  - {self.name}이(가) {actual_damage} 피해를 받았다. (HP {self.hp}/{self.max_hp})")
        return actual_damage

    def attack_target(self, target):
        damage = random.randint(1, self.attack)

        is_crit = self._try_crit()
        if is_crit:
            damage = int(damage * 1.8)
            print(f"{self.name}의 공격! (치명타!) -> 기본 피해 {damage}")
        else:
            print(f"{self.name}의 공격! -> 기본 피해 {damage}")

        target.take_damage(damage)


class Player(Character):
    def __init__(self, name):
        super().__init__(
            name=name,
            level=1,
            hp=100,
            attack=25,
            defense=5,
            crit=0.12,
            evasion=0.08
        )
        self.exp = 0
        self.potions = 3  # 시작 포션 수

        # 스킬 쿨다운(턴 단위)
        self.cooldowns = {"power_strike": 0}

    def gain_experience(self, amount):
        self.exp += amount
        print(f"[EXP] {amount} 획득 (현재 EXP: {self.exp})")

    def level_up(self):
        leveled = False
        while self.exp >= 500:
            self.exp -= 500
            self.level += 1

            # 성장
            self.attack += 10
            self.defense += 5
            self.max_hp += 15
            self.hp = min(self.max_hp, self.hp + 20)  # 레벨업 시 약간 회복

            leveled = True
            print(f"[LEVEL UP] 레벨 {self.level} 달성! (ATK+10, DEF+5, MAXHP+15)")

        return leveled

    def rest_after_battle(self):
        # 전투 후 숨 고르기: 체력 20% 회복
        heal = int(self.max_hp * 0.2)
        self.hp = min(self.max_hp, self.hp + heal)
        print(f"[REST] 전투 후 회복: +{heal} (HP {self.hp}/{self.max_hp})")

    def use_potion(self):
        if self.potions <= 0:
            print("[ITEM] 포션이 없다.")
            return False

        heal = 35
        before = self.hp
        self.hp = min(self.max_hp, self.hp + heal)
        self.potions -= 1
        print(f"[ITEM] 포션 사용: HP {before}->{self.hp} (남은 포션 {self.potions})")
        return True

    def tick_cooldowns(self):
        for k in self.cooldowns:
            if self.cooldowns[k] > 0:
                self.cooldowns[k] -= 1

    # 스킬 1: 강공
    def power_strike(self, target):
        if self.cooldowns["power_strike"] > 0:
            print(f"[SKILL] 강공은 쿨다운 중이다. (남은 턴 {self.cooldowns['power_strike']})")
            return False

        base = random.randint(int(self.attack * 0.8), int(self.attack * 1.3))
        damage = int(base * 2.0)
        print(f"{self.name}의 [강공]! -> 기본 피해 {damage} (쿨다운 2턴)")
        target.take_damage(damage)

        self.cooldowns["power_strike"] = 2
        return True

    # 스킬 2: 방어 태세
    def guard(self):
        self.guard_until_next_turn = True
        print(f"{self.name}은(는) [방어 태세]를 취했다! (다음 피격 피해 감소)")
        return True

    def choose_action(self, monster):
        """
        입력 기반(수동)으로 선택.
        바이브 코딩용으로 UI를 간단히 했고, 자동 선택으로 바꾸고 싶으면 여기만 수정하면 됨.
        """
        print("\n[행동 선택]")
        print("1) 공격")
        print("2) 강공(쿨다운 2)")
        print("3) 방어 태세")
        print("4) 포션 사용")
        choice = input("선택: ").strip()

        if choice == "1":
            self.attack_target(monster)
            return
        if choice == "2":
            ok = self.power_strike(monster)
            if not ok:
                self.attack_target(monster)
            return
        if choice == "3":
            self.guard()
            return
        if choice == "4":
            used = self.use_potion()
            if not used:
                self.attack_target(monster)
            return

        # 잘못 입력하면 기본 공격
        print("[WARN] 잘못된 입력. 기본 공격으로 진행.")
        self.attack_target(monster)


class Monster(Character):
    def __init__(self, name, level):
        hp = random.randint(10, 30) * level
        attack = random.randint(5, 20) * level
        defense = random.randint(1, 5) * level

        # 몬스터는 레벨에 따라 약간 더 위협적이 되도록 조정
        crit = min(0.03 + 0.01 * level, 0.12)
        evasion = min(0.02 + 0.005 * level, 0.08)

        super().__init__(name=name, level=level, hp=hp, attack=attack, defense=defense, crit=crit, evasion=evasion)


# -----------------------------
# Battle System
# -----------------------------
def battle(player, monster):
    print("\n" + "=" * 40)
    print(f"[BATTLE START] {monster.name} (Lv {monster.level}) 등장!")
    print(f"{monster.name} 스탯: HP {monster.hp}/{monster.max_hp}, ATK {monster.attack}, DEF {monster.defense}")
    print("=" * 40)

    # 전투 턴 루프
    while player.is_alive() and monster.is_alive():
        # 플레이어 턴
        player.choose_action(monster)
        if not monster.is_alive():
            break

        # 몬스터 턴
        monster.attack_target(player)

        # 턴 종료 처리
        player.tick_cooldowns()

    if player.is_alive():
        exp_gain = monster.level * 20
        player.gain_experience(exp_gain)
        player.level_up()
        print("[RESULT] 전투 승리!")
        return True
    else:
        print("[RESULT] 전투 패배...")
        return False


# -----------------------------
# Main
# -----------------------------
def main():
    monster_dict = {"슬라임": 1, "고블린": 2, "오크": 3}

    player_name = input("플레이어의 이름을 입력하세요: ").strip()
    if not player_name:
        player_name = "히어로"

    player = Player(player_name)
    print(f"\n[PLAYER] {player.name} 생성 완료! (Lv {player.level}, HP {player.hp}/{player.max_hp}, ATK {player.attack}, DEF {player.defense}, 포션 {player.potions})")

    for monster_name, monster_level in monster_dict.items():
        monster = Monster(monster_name, monster_level)
        win = battle(player, monster)

        if not win:
            print("게임오버")
            break
        else:
            print(f"{monster.name}을(를) 이겼다!")
            player.rest_after_battle()
            print(f"[STATUS] {player.name}: Lv {player.level}, HP {player.hp}/{player.max_hp}, ATK {player.attack}, DEF {player.defense}, 포션 {player.potions}")

    if player.is_alive():
        print("\n[ENDING] 모든 몬스터를 처치했다! 승리!")


if __name__ == "__main__":
    main()


플레이어의 이름을 입력하세요: 0000

[PLAYER] 0000 생성 완료! (Lv 1, HP 100/100, ATK 25, DEF 5, 포션 3)

[BATTLE START] 슬라임 (Lv 1) 등장!
슬라임 스탯: HP 20/20, ATK 17, DEF 4

[행동 선택]
1) 공격
2) 강공(쿨다운 2)
3) 방어 태세
4) 포션 사용
선택: 1
0000의 공격! -> 기본 피해 7
  - 슬라임이(가) 회피했다!
슬라임의 공격! -> 기본 피해 17
  - 0000이(가) 12 피해를 받았다. (HP 88/100)

[행동 선택]
1) 공격
2) 강공(쿨다운 2)
3) 방어 태세
4) 포션 사용
선택: 2
0000의 [강공]! -> 기본 피해 54 (쿨다운 2턴)
  - 슬라임이(가) 50 피해를 받았다. (HP 0/20)
[EXP] 20 획득 (현재 EXP: 20)
[RESULT] 전투 승리!
슬라임을(를) 이겼다!
[REST] 전투 후 회복: +20 (HP 100/100)
[STATUS] 0000: Lv 1, HP 100/100, ATK 25, DEF 5, 포션 3

[BATTLE START] 고블린 (Lv 2) 등장!
고블린 스탯: HP 56/56, ATK 10, DEF 4

[행동 선택]
1) 공격
2) 강공(쿨다운 2)
3) 방어 태세
4) 포션 사용
선택: 2
[SKILL] 강공은 쿨다운 중이다. (남은 턴 2)
0000의 공격! -> 기본 피해 14
  - 고블린이(가) 10 피해를 받았다. (HP 46/56)
고블린의 공격! -> 기본 피해 9
  - 0000이(가) 4 피해를 받았다. (HP 96/100)

[행동 선택]
1) 공격
2) 강공(쿨다운 2)
3) 방어 태세
4) 포션 사용
선택: 2
[SKILL] 강공은 쿨다운 중이다. (남은 턴 1)
0000의 공격! -> 기본 피해 13
  - 고블린이(가) 9 피해를 받았다. (HP 37/56)
고블린의 공격! -> 기본 피해 8
  - 0000이(가) 3 피해를 받았다. (HP 9

다음 액션 (선택만 하면 됨)

아래 중 하나만 지시하면, 그 방향으로 바로 추가 업그레이드 버전을 내겠습니다.

자동 전투(Auto Battle): 입력 없이 AI가 최적 행동 선택

아이템 드롭/상점: 몬스터 처치 후 포션/장비 획득, 구매

장비 시스템: 무기/방어구 착용으로 스탯 변화

상태이상: 출혈/중독/기절 같은 디버프

스테이지/보스: 3마리 고정이 아니라, 웨이브 + 보스전